In [83]:
# https://www.kaggle.com/c/california-house-price/overview
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [75]:
housing_test = pd.read_csv('housing_test.csv', index_col='Id')
housing_train = pd.read_csv('housing_train.csv')
housing_test.shape, housing_train.shape

((6192, 9), (14448, 10))

In [76]:
housing_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
Id,,,,,,,,,
0,-122.64,38.24,40.0,1974.0,410.0,1039.0,398.0,3.7917,<1H OCEAN
1,-119.05,36.09,9.0,3297.0,568.0,1749.0,568.0,4.0217,INLAND
2,-116.98,32.85,12.0,3570.0,713.0,3321.0,666.0,4.0882,<1H OCEAN
3,-118.42,34.20,27.0,3201.0,970.0,3403.0,948.0,2.2377,<1H OCEAN
4,-118.47,34.01,41.0,2704.0,557.0,1047.0,478.0,4.4211,<1H OCEAN


In [78]:
housing_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-116.46,33.82,6.0,4863.0,920.0,3010.0,828.0,3.9508,104200.0,INLAND
1,-117.04,34.00,21.0,4624.0,852.0,2174.0,812.0,3.5255,132100.0,INLAND
2,-121.03,37.55,32.0,946.0,198.0,624.0,173.0,1.9728,97900.0,INLAND
3,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
4,-122.26,37.83,52.0,1656.0,420.0,718.0,382.0,2.6768,182300.0,NEAR BAY


In [79]:
housing_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14448 entries, 0 to 14447
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           14448 non-null  float64
 1   latitude            14448 non-null  float64
 2   housing_median_age  14448 non-null  float64
 3   total_rooms         14448 non-null  float64
 4   total_bedrooms      14310 non-null  float64
 5   population          14448 non-null  float64
 6   households          14448 non-null  float64
 7   median_income       14448 non-null  float64
 8   median_house_value  14448 non-null  float64
 9   ocean_proximity     14448 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.1+ MB


In [80]:
housing_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6192 entries, 0 to 6191
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           6192 non-null   float64
 1   latitude            6192 non-null   float64
 2   housing_median_age  6192 non-null   float64
 3   total_rooms         6192 non-null   float64
 4   total_bedrooms      6123 non-null   float64
 5   population          6192 non-null   float64
 6   households          6192 non-null   float64
 7   median_income       6192 non-null   float64
 8   ocean_proximity     6192 non-null   object 
dtypes: float64(8), object(1)
memory usage: 483.8+ KB


### Кодируем категориальную переменную, заменяем неопределенные значения на медиану

In [41]:
#кодируем
housing_train_encoder = pd.get_dummies(housing_train, drop_first = True)
housing_test_encoder = pd.get_dummies(housing_test, drop_first = True)
housing_test_encoder['total_bedrooms'].fillna(housing_test_encoder['total_bedrooms'].median(), inplace=True)
housing_train_encoder['total_bedrooms'].fillna(housing_train_encoder['total_bedrooms'].median(), inplace=True)
housing_test_encoder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6192 entries, 0 to 6191
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitude                   6192 non-null   float64
 1   latitude                    6192 non-null   float64
 2   housing_median_age          6192 non-null   float64
 3   total_rooms                 6192 non-null   float64
 4   total_bedrooms              6192 non-null   float64
 5   population                  6192 non-null   float64
 6   households                  6192 non-null   float64
 7   median_income               6192 non-null   float64
 8   ocean_proximity_INLAND      6192 non-null   uint8  
 9   ocean_proximity_ISLAND      6192 non-null   uint8  
 10  ocean_proximity_NEAR BAY    6192 non-null   uint8  
 11  ocean_proximity_NEAR OCEAN  6192 non-null   uint8  
dtypes: float64(8), uint8(4)
memory usage: 459.6 KB


In [81]:
X = housing_train_encoder.drop(['median_house_value'], axis = 1)
y = housing_train_encoder['median_house_value']

Делаем предсказание с помощью линейной регрессии

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
reg = LinearRegression()

X_scaled = scaler.fit_transform(X)
reg.fit(X_scaled, y)

housing_test_scaled = scaler.fit_transform(housing_test_encoder)
prediction = reg.predict(housing_test_scaled)
prediction

array([246576.40378021, 133832.31595407, 160637.27256709, ...,
       268228.09550495, 273792.05718079, 251313.39913305])

Записываем прогноз в файл

In [44]:
result = pd.DataFrame({'Id': housing_test.reset_index()['Id'], "median_house_value": prediction})
result.to_csv('prediction/LinearRegression.csv', index=False)

In [60]:
print("{0} - min".format(housing_train['median_house_value'].min()))
print("{0} - max".format(housing_train['median_house_value'].max()))
print("{0} - median".format(housing_train['median_house_value'].median()))

14999.0 - min
500001.0 - max
180300.0 - median


#### Score на Kaggle - 69902.21138(результат вписывается в разброс)
